In [ ]:
# Copyright 2020 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deploying NVIDIA Triton Inference Server in AI Platform Prediction Custom Container (Google Cloud SDK)

In this notebook, we will walk through the process of deploying NVIDIA's Triton Inference Server into AI Platform Prediction Custom Container service in the Direct Model Server mode:

![](img/caip_triton_container_diagram_direct.jpg)


In [ ]:
PROJECT_ID='[Enter project name - REQUIRED]'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

In [ ]:
import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)

In [ ]:
'''
# Test values

PROJECT_ID='tsaikevin-1236'
REPOSITORY='caipcustom'
REGION='us-central1'
TRITON_VERSION='20.06'

import os
import random
import requests
import json

MODEL_BUCKET='gs://{}-{}'.format(PROJECT_ID,random.randint(10000,99999))
ENDPOINT='https://{}-ml.googleapis.com/v1'.format(REGION)
TRITON_IMAGE='tritonserver:{}-py3'.format(TRITON_VERSION)
CAIP_IMAGE='{}-docker.pkg.dev/{}/{}/{}'.format(REGION,PROJECT_ID,REPOSITORY,TRITON_IMAGE)
'''

In [ ]:
!gcloud config set project $PROJECT_ID

### Create the Artifact Registry
This will be used to store the container image for the model server Triton.

In [ ]:
!gcloud beta artifacts repositories create $REPOSITORY --repository-format=docker --location=$REGION

In [ ]:
!gcloud beta auth configure-docker $REGION-docker.pkg.dev --quiet

### Prepare the container
We will make a copy of the Triton container image into the Artifact Registry, where AI Platform Custom Container Prediction will only pull from during Model Version setup. The following steps will download the NVIDIA Triton Inference Server container to your VM, then upload it to your repo.

In [ ]:
!docker pull nvcr.io/nvidia/$TRITON_IMAGE && \
 docker tag nvcr.io/nvidia/$TRITON_IMAGE $CAIP_IMAGE && \
 docker push $CAIP_IMAGE

### Prepare model Artifacts
Clone the NVIDIA Triton Inference Server repo.

In [ ]:
!git clone -b r$TRITON_VERSION https://github.com/triton-inference-server/server.git

Create the GCS bucket where the model artifacts will be copied to.

In [ ]:
!gsutil mb $MODEL_BUCKET

Stage model artifacts and copy to bucket.

In [ ]:
!mkdir model_repository

In [ ]:
!cp -R server/docs/examples/model_repository/* model_repository/

In [ ]:
!./server/docs/examples/fetch_models.sh

In [ ]:
!gsutil -m cp -R model_repository/ $MODEL_BUCKET

In [ ]:
!gsutil ls -RLl $MODEL_BUCKET/model_repository

### Prepare request payload

To prepare the payload format, we have included a utility get_request_body_simple.py.  To use this utility, install the following library:

In [ ]:
!pip3 install geventhttpclient

#### Prepare non-binary request payload

The first model will illustrate a non-binary payload.  The following command will create a KF Serving v2 format non-binary payload to be used with the "simple" model:

In [ ]:
!python3 get_request_body_simple.py -m simple

#### Prepare binary request payload

Triton's implementation of KF Serving v2 protocol for binary data appends the binary data after the json body.  Triton requires an additional header for offset:

`Inference-Header-Content-Length: [offset]`

We have provided a script that will automatically resize the image to the proper size for ResNet-50 [224, 224, 3] and calculate the proper offset.  The following command takes an image file and outputs the necessary data structure to be use with the "resnet50_netdef" model.  Please note down this offset as it will be used later.

In [ ]:
!python3 get_request_body_simple.py -m image -f server/qa/images/mug.jpg

## Create and deploy Model and Model Version

In this section, we will deploy two models:
1. Simple model with non-binary data.  KF Serving v2 protocol specifies a json format with non-binary data in the json body itself.
2. Binary data model with ResNet-50.  Triton's implementation of binary data for KF Server v2 protocol.


### Simple model (non-binary data)

#### Create Model

AI Platform Prediction uses a Model/Model Version Hierarchy, where the Model is a logical grouping of Model Versions.  We will first create the Model.

Because the MODEL_NAME variable will be used later to specify the predict route, and Triton will use that route to run prediction on a specific model, we must set the value of this variable to a valid name of a model.  For this section, will use the "simple" model.

In [ ]:
MODEL_NAME='simple'

In [ ]:
!gcloud ai-platform models create $MODEL_NAME --region $REGION --enable-logging

In [ ]:
!gcloud ai-platform models list --region $REGION

#### Create Model Version

After the Model is created, we can now create a Model Version under this Model.  Each Model Version will need a name that is unique within the Model.  In AI Platform Prediction Custom Container, a {Project}/{Model}/{ModelVersion} uniquely identifies the specific container and model artifact used for inference.

In [ ]:
VERSION_NAME='v01'

The following config file will be used in the Model Version creation command.

#### Command with YAML config file

In [ ]:
import yaml

config_simple={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_simple.yaml', 'w') as file:
    config = yaml.dump(config_simple, file)

In [ ]:
!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_simple.yaml \
  --region=$REGION \
  --async

#### To see details of the Model Version just created

In [ ]:
!gcloud ai-platform versions describe $VERSION_NAME --model=$MODEL_NAME --region=$REGION

#### To list all Model Versions and their states in this Model

In [ ]:
!gcloud ai-platform versions list --model=$MODEL_NAME --region=$REGION

#### Run prediction using `curl`

The "simple" model takes two tensors with shape [1,16] and does a couple of basic arithmetic operation.

In [ ]:
!curl -X POST $ENDPOINT/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$VERSION_NAME:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -d @simple.json

In [ ]:
# this doesn't: gcloud auth application-default print-access-token

!curl -X POST $ENDPOINT/projects/$PROJECT_ID/models/$MODEL_NAME/versions/$VERSION_NAME:predict \
    -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth application-default print-access-token`" \
    -d @simple.json

#### Run prediction using Using `requests` library

In [ ]:
with open('simple.json', 'r') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, MODEL_NAME, VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

In [ ]:
# this doesn't work: gcloud auth application-default print-access-token

with open('simple.json', 'r') as s:
    data=s.read()
    
PREDICT_URL = "https://us-central1-ml.googleapis.com/v1/projects/tsaikevin-1236/models/simple/versions/v01:predict"
HEADERS = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

### ResNet-50 model (binary data)

#### Create Model

In [ ]:
BINARY_MODEL_NAME='resnet50_netdef'

In [ ]:
!gcloud ai-platform models create $BINARY_MODEL_NAME --region $REGION --enable-logging

#### Create Model Version

In [ ]:
BINARY_VERSION_NAME='v01'

#### Command with YAML config file

In [ ]:
import yaml

config_binary={'deploymentUri': MODEL_BUCKET+'/model_repository', \
               'container': {'image': CAIP_IMAGE, \
                             'args': ['tritonserver', '--model-repository=$(AIP_STORAGE_URI)'], \
                             'env': [], \
                             'ports': {'containerPort': 8000}}, \
               'routes': {'predict': '/v2/models/'+BINARY_MODEL_NAME+'/infer', \
                          'health': '/v2/models/'+BINARY_MODEL_NAME}, \
               'machineType': 'n1-standard-4', 'autoScaling': {'minNodes': 1}}

with open(r'config_binary.yaml', 'w') as file:
    config_binary = yaml.dump(config_binary, file)

In [ ]:
!gcloud beta ai-platform versions create $BINARY_VERSION_NAME \
  --model $BINARY_MODEL_NAME \
  --accelerator count=1,type=nvidia-tesla-t4 \
  --config config_binary.yaml \
  --region=$REGION \
  --async

#### To see details of the Model Version just created

In [ ]:
!gcloud ai-platform versions describe $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION

#### To list all Model Versions and their states in this Model

In [ ]:
!gcloud ai-platform versions list --model=$BINARY_MODEL_NAME --region=$REGION

#### Run prediction using `curl`

Recall the offset value calcuated above.  The binary case has an additional header:

`Inference-Header-Content-Length: [offset]`

In [ ]:
!curl --request POST $ENDPOINT/projects/$PROJECT_ID/models/$BINARY_MODEL_NAME/versions/$BINARY_VERSION_NAME:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat

In [ ]:
# this doesn't work: gcloud auth application-default print-access-token

!curl --request POST $ENDPOINT/projects/$PROJECT_ID/models/$BINARY_MODEL_NAME/versions/$BINARY_VERSION_NAME:predict \
    -k -H "Content-Type: application/octet-stream" \
    -H "Authorization: Bearer `gcloud auth application-default print-access-token`" \
    -H "Inference-Header-Content-Length: 138" \
    --data-binary @payload.dat

#### Run prediction using Using `requests` library

In [ ]:
with open('payload.dat', 'rb') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, BINARY_MODEL_NAME, BINARY_VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Inference-Header-Content-Length': '138',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

In [ ]:
# this doesn't work: gcloud auth application-default print-access-token

with open('payload.dat', 'rb') as s:
    data=s.read()
    
PREDICT_URL = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_ID, BINARY_MODEL_NAME, BINARY_VERSION_NAME)
HEADERS = {
  'Content-Type': 'application/octet-stream',
  'Inference-Header-Content-Length': '138',
  'Authorization': 'Bearer {}'.format(os.popen('gcloud auth application-default print-access-token').read().rstrip())
}

response = requests.request("POST", PREDICT_URL, headers=HEADERS, data = data).content.decode()

json.loads(response)

## Clean up

In [ ]:
!gcloud ai-platform versions delete $VERSION_NAME --model=$MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform models delete $MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform versions delete $BINARY_VERSION_NAME --model=$BINARY_MODEL_NAME --region=$REGION --quiet

In [ ]:
!gcloud ai-platform models delete $BINARY_MODEL_NAME --region=$REGION --quiet

In [ ]:
!gsutil -m rm -r -f $MODEL_BUCKET

In [ ]:
!rm -rf model_repository triton-inference-server server *.yaml *.dat *.json